# ***Segmenation and Heatmap***

In [5]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Input, Dense, BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Dropout, Lambda
from keras.layers import concatenate
from keras.layers.pooling import MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
from skimage.io import imread, imshow
from skimage.transform import resize
from scipy import ndimage

from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

tf.version.VERSION

'2.11.0'

In [ ]:
IMG_HEIGHT=1024
IMG_WIDTH=1024
IMG_CHANNELS=3

Seg_path='path to your the segmentation model/'
model = load_model(Seg_path+'/SeedSeg-tf11.h5')

## preprocess the image for segmentation 
## the size should be: 4032, 3024, 3
im_path='Enter The path to your image/'
img   = cv2.imread(im_path+'/IMG_0864.JPG')
img   = img[int((img.shape[0]-2000)/2):int(img.shape[0]-(img.shape[0]-2000)/2)\
                 ,int((img.shape[1]-2000)/2):int(img.shape[1]-(img.shape[1]-2000)/2)\
                 ,:] 

img_n = np.expand_dims(cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA),axis=-1)

##predict the seed region 
preds_test    = model.predict(np.expand_dims(img_n, axis=0), verbose=1)
predicted_img = np.reshape(preds_test, (IMG_HEIGHT,IMG_WIDTH))
nrn           = resize(np.squeeze(predicted_img),(img.shape[0],img.shape[1]), mode='constant', preserve_range=True)
nrn           = (nrn > 0.70).astype(np.uint8)
mask          = nrn*255
#plt.figure()
#plt.imshow(mask) 
#plt.show() 

## generate heatmap
img[np.where(mask==255)] = 225
heatmap_img              = cv2.applyColorMap(img[:,:,0], cv2.COLORMAP_JET)
#plt.figure()
#plt.imshow(mask) 
#plt.show() 

#save image 
save_path= 'path where to save the segmented image/'
cv2.imwrite(save_path+'/Heatmap.jpg',heatmap_img)

# ***TKW prediction***

In [ ]:
import os
import random
import numpy as np
from skimage.io import imread
from skimage.transform import resize 

import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import  Dropout, Flatten, Dense, GlobalAveragePooling2D,AveragePooling2D,BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
img_width   = 224
img_height  = 224

img= imread(save_path+'/Heatmap.jpg')
img = np.expand_dims(resize(img, (img_height, img_width), mode='constant', preserve_range=True),axis=-1)
X_Im = img[:,:,:,0][np.newaxis,...]/255

model_path='path to the TKW model/'
SavedModel = load_model(model_path+'/ResNet50-TKW-all.h5') 

PRED_train = SavedModel.predict(X_Im)
PRED_train = np.around(PRED_train,3)
print(PRED_train)